In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p polars,instructor,openai,ollama

Author: Prayson W. Daniel

Last updated: 2024-12-17T12:00:57.051725+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

polars    : 1.7.1
instructor: 1.4.3
openai    : 1.57.4
ollama    : 0.3.3

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
from typing import Final, Literal

import instructor
import polars as pl
from openai import OpenAI
from pydantic import BaseModel, Field

In [3]:
MODEL: Final = "gemma2"

VALID_EMOTIONS: tuple[str, ...] = tuple(
    {
        "admiration",
        "approval",
        "gratitude",
        "optimism",
        "joy",
        "neutral",
        "disapproval",
        "disappointment",
        "love",
        "relief",
        "excitement",
        "desire",
        "caring",
        "pride",
        "realization",
        "disgust",
        "annoyance",
        "remorse",
        "amusement",
        "anger",
        "grief",
        "curiosity",
        "fear",
        "surprise",
        "sadness",
        "embarrassment",
        "confusion",
        "nervousness",
    }
)

class Emotion(BaseModel):
    kind: Literal[VALID_EMOTIONS]  # type: ignore # literal does not like dynamic values
    reason: str = Field(
        ..., description="A short third person reason for selected the emotion"
    )

In [4]:
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NotNeeded",
    ),
    mode=instructor.Mode.JSON,
)

In [5]:
def classifier(text: str, model: str = MODEL) -> Emotion:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": ("You are a brilliant emotion pyschologist."
                            f"What is the text emotion: Text: {text}"),
            }
        ],
        response_model=Emotion,
        temperature=0.0,
    )

    return response

In [6]:
pl.Config.set_fmt_str_lengths(100).set_tbl_rows(
    -1
)  # show more text. default is 25, show all rowss

dataf = pl.read_json("../data/examples.json").explode(["emotion", "text"])

In [8]:
dataf.sample(3, seed=7)

emotion,text
str,str
"""confusion""","""The situation is so unclear right now, it’s leaving me confused."""
"""love""","""My love for nature grows with every hike in the mountains."""
"""caring""","""It’s important to care for your mental health as much as your physical health."""


In [9]:
classifier("I dont' know how to think about where the world is going?")

Emotion(kind='confusion', reason='The speaker expresses uncertainty and lack of direction regarding the future state of the world.')

In [10]:
(
    dataf.sample(8, seed=42)
    .with_columns(
        pl.col("text")
        .map_elements(classifier, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.kind, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reason, return_dtype=pl.String)
        .alias("reason"),
    )
)

emotion,text,prediction,reason
str,str,str,str
"""confusion""","""His mixed signals have left me feeling utterly confused.""","""confusion""","""His mixed signals have left him feeling uncertain and perplexed."""
"""joy""","""I can’t stop smiling after watching that movie. Such a feel-good story!""","""joy""","""The movie was a feel-good story."""
"""disappointment""","""I’m disappointed that the event was canceled last minute.""","""disappointment""","""The event was canceled last minute."""
"""anger""","""I’m furious that the package didn’t arrive on time.""","""anger""","""The package didn't arrive on time."""
"""realization""","""It’s in moments like these that you realize what really matters in life.""","""realization""","""The text suggests a moment of introspection and understanding about life's priorities."""
"""love""","""My love for nature grows with every hike in the mountains.""","""love""","""The speaker expresses a deepening affection for nature due to their hiking experiences."""
"""caring""","""She’s always caring for others, making sure everyone feels valued.""","""caring""","""She consistently prioritizes the well-being and feelings of others."""
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!""","""admiration""","""He accomplished a challenging feat despite an injury."""
